<a href="https://colab.research.google.com/github/kotaxtech/Deeplabv3plus/blob/main/deeplabv3plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

まず、Tensorflowのインストール

Versionが1.15.2であることを確認した後、GPU版のTensorflowもインストールします。

※下のように警告文（Warning）が出ることがありますが、スルーで大丈夫です。

```
WARNING: The following packages were previously imported in this runtime:

  [gast,tensorflow,tensorflow_core]

You must restart the runtime in order to use newly installed versions.
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip3 install  tensorflow-gpu==1.15.2
!pip install tensorflow-determinism
!pip install tf_slim
import tensorflow as tf
print(tf.__version__)

次に、Githubからモデルをインストールしていきます。

（Tensorflowの公式リポジトリに専用のモデルがあるようです。）

model_test.pyを実行し、上手くいけば"OK"と出力されます。

In [ ]:
%%bash
cd /content/drive/My Drive/
mkdir tensorflow
cd tensorflow
git clone https://github.com/tensorflow/models.git

cd /content/drive/My Drive/tensorflow/models/research/
researchpath = %pwd
env PYTHONPATH = $researchpath:$researchpath/slim
#export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
python deeplab/model_test.py

# 教師データの変換

教師データをグレースケールおよびTFrecordに変換します．まず/tensorflow/models/research/deeplab/datasetsに"pascal_voc_seg"という名前でディレクトリを作成します．(以下のコードを実行すると勝手に作成されます．)

In [ ]:
CURRENT_DIR = %pwd
WORK_DIR = "./pascal_voc_seg"
!mkdir -v $WORK_DIR
%cd $WORK_DIR

さらにpascal_voc_segディレクトリ直下に各ディレクトリを作成します．(これも以下のコードを実行すると勝手に作成されます．)

In [ ]:
%cd $CURRENT_DIR

PASCAL_ROOT = WORK_DIR + "/VOCdevkit/VOC2012"

SEG_FOLDER = PASCAL_ROOT + "/SegmentationClass"
SEMANTIC_SEG_FOLDER = PASCAL_ROOT + "/SegmentationClassRaw"

IMAGE_FOLDER = PASCAL_ROOT + "/JPEGImages"
LIST_FOLDER = PASCAL_ROOT + "/ImageSets/Segmentation"

!mkdir -p -v $PASCAL_ROOT $LIST_FOLDER
!mkdir -v $SEG_FOLDER  $SEMANTIC_SEG_FOLDER $IMAGE_FOLDER

作成されたSegmentationディレクトリ内にtrain.txt，val.txtを作成します．

（⬆これは、事前に作成する訓練データと検証データのファイル名です．）

学習データと教師データの名前は同じにします．

このとき，学習データをjpg 教師データをpngの拡張子にします．（そういう仕様？）

ex)学習データ：001.jpg トレーニングデータ：001.png

以下のようなフォルダ構成になっているか確認してください。

```
./pascal_voc_seg
  └ VOCdevkit
    └ VOC2012
        ├ ImageSets
        │  └ Segmentation
        │     ├ train.txt(訓練データのリスト)
        │     └ val.txt(検証データのリスト)
        ├ SegmentationClass(教師データ)
        ├ SegmentationClassRaw(教師データのGrayScale版：この時点では空)
        └ JPEGImages(元データ)

```


次に/tensorflow/models/research/deeplab/datasets/data_generator.py の

82～91行目を自分のデータセットに合わせて書き換えます。

 [コチラ](https://qiita.com/mucchyo/items/d21993abee5e6e44efad#%E5%AD%A6%E7%BF%92%E6%A4%9C%E8%A8%BC%E3%83%87%E3%83%BC%E3%82%BF%E8%A8%AD%E5%AE%9A%E3%81%AE%E5%A4%89%E6%9B%B4)を参考にしました。

※注意
- train, valはそれぞれ訓練データ枚数，検証データ枚数
- trainvalは...train + valの値を設定
- train_augは、画像データの拡張枚数。そのままでいい（らしい…）
- 今回は転移学習(21クラス→3クラス)を実施するので、num_classesは21のままでいい（らしい…）

```
_PASCAL_VOC_SEG_INFORMATION = DatasetDescriptor(
    splits_to_sizes={
        'train': 600,
        'train_aug': 10582,
        'trainval': 975,
        'val': 375,
    },
    num_classes=21,
    ignore_label=255,
)
```

現段階で教師データはインデックスカラー形式になっているので、グレースケールに変換する必要があります。（…あるのか？）

以下のコードを実行すると、SegmentationClassRawフォルダにグレースケール画像が出力されます。


In [ ]:
print("Removing the color map in ground truth annotations...")

%time !python remove_gt_colormap.py \
  --original_gt_folder=$SEG_FOLDER \
  --output_dir=$SEMANTIC_SEG_FOLDER

データセットの事前準備できたので、次はTensorFlow用の形式である「TFRecord」に変換していきます。

従来よりもデータサイズが削減できて、学習コストを抑えることができるそう…

以下のコードを実行したらpascal_voc_segフォルダに作成されているtfrecordフォルダを確認してください。

うまく動作していると以下のファイルが出力されています。

- train-00000-of-00004.tfrecord  
- train-00001-of-00004.tfrecord  
- train-00002-of-00004.tfrecord
- train-00003-of-00004.tfrecord
- val-00000-of-00004.tfrecord
- val-00001-of-00004.tfrecord
- val-00002-of-00004.tfrecord
- val-00003-of-00004.tfrecord


In [ ]:
#OUTPUT_DIR = WORK_DIR + "/tfrecord"
OUTPUT_DIR = "./pascal_voc_seg/tfrecord"
!mkdir -p $OUTPUT_DIR

#IMAGE_FOLDER = PASCAL_ROOT + "/JPEGImages"
#LIST_FOLDER = PASCAL_ROOT + "/ImageSets/Segmentation"

print("Converting PASCAL VOC 2012 dataset...")

%time !python build_voc2012_data.py \
  --image_folder=$IMAGE_FOLDER \
  --semantic_segmentation_folder=$SEMANTIC_SEG_FOLDER \
  --list_folder=$LIST_FOLDER \
  --image_format="jpg" \
  --output_dir=$OUTPUT_DIR

# ディレクトリの作成

ソースコード上で使用するディレクトリを設定する．(以下のコードを実行すると勝手にディレクトリが作成される．)

|フォルダ名|意味|
|---|---|
|INIT_FOLDER|学習済みの結合荷重|
|TRAIN_LOGDIR|学習結果(訓練)の保存先|
|EVAL_LOGDIR|学習結果(検証)の保存先|
|VIS_LOGDIR|可視化画像(検証)の保存先|
|EXPORT_DIR|学習したモデルの保存先|
|PASCAL_DATASET|TFRecord化したデータセット|

In [ ]:
CURRENT_DIR = %pwd
DATASET_DIR = "datasets"

PASCAL_FOLDER = "/pascal_voc_seg"
EXP_FOLDER = "/exp/train_on_trainval_set"
INIT_FOLDER = DATASET_DIR + PASCAL_FOLDER + "/init_models"
TRAIN_LOGDIR = DATASET_DIR + PASCAL_FOLDER + EXP_FOLDER + "/train"
EVAL_LOGDIR = DATASET_DIR + PASCAL_FOLDER + EXP_FOLDER + "/eval"
VIS_LOGDIR = DATASET_DIR + PASCAL_FOLDER + EXP_FOLDER + "/vis"
EXPORT_DIR = DATASET_DIR + PASCAL_FOLDER + EXP_FOLDER + "/export"

PASCAL_DATASET = DATASET_DIR + PASCAL_FOLDER + "/tfrecord"

!mkdir -p -v $INIT_FOLDER $TRAIN_LOGDIR $EVAL_LOGDIR $VIS_LOGDIR $EXPORT_DIR

学習済み結合荷重をダウンロードする．(1回ダウンロードしたら2周目以降は実行しなくて大丈夫．)

In [ ]:
TF_INIT_ROOT = "http://download.tensorflow.org/models"
TF_INIT_CKPT = "deeplabv3_pascal_train_aug_2018_01_04.tar.gz"
%cd $INIT_FOLDER
!wget -nd -c $TF_INIT_ROOT/$TF_INIT_CKPT
!tar -xf $TF_INIT_CKPT

TF_INIT_ROOT = "http://download.tensorflow.org/models"
TF_INIT_CKPT = "deeplabv3_xception_2018_01_04.tar.gz"
%cd $INIT_FOLDER
!wget -nd -c $TF_INIT_ROOT/$TF_INIT_CKPT
!tar -xf $TF_INIT_CKPT

TF_INIT_ROOT = "http://download.tensorflow.org/models"
TF_INIT_CKPT = "xception_65_coco_pretrained_2018_10_02.tar.gz"
%cd $INIT_FOLDER
!wget -nd -c $TF_INIT_ROOT/$TF_INIT_CKPT
!tar -xf $TF_INIT_CKPT
%cd $CURRENT_DIR

指定したディレクトリに3つの学習済み結合荷重がダウンロードされていることを確認する．

In [ ]:
!ls $INIT_FOLDER
'''
出力結果…
　deeplabv3_pascal_train_aug
　deeplabv3_pascal_train_aug_2018_01_04.tar.gz
　deeplabv3_xception_2018_01_04.tar.gz
　xception
　xception_65_coco_pretrained
　xception_65_coco_pretrained_2018_10_02.tar.gz
'''

# 学習

ここから学習です．
models/research/deeplab/train.pyの266行目を書き換えると学習時の重みが変更できます．


```
  for output, num_classes in six.iteritems(outputs_to_num_classes):
    train_utils.add_softmax_cross_entropy_loss_for_each_scale(
        outputs_to_scales_to_logits[output],
        samples[common.LABEL],
        num_classes,
        ignore_label,
        #loss_weight=model_options.label_weights,
        loss_weight=[0.002, 2.63, 2.56],←ココ
        upsample_logits=FLAGS.upsample_logits,
        hard_example_mining_step=FLAGS.hard_example_mining_step,
        top_k_percent_pixels=FLAGS.top_k_percent_pixels,
        scope=output)
```

In [ ]:
# 学習回数を指定(デフォルトでは30000 iterations)
NUM_ITERATIONS=30000

主に変更したのは，以下の4つ．
- training_number_of_step：学習回数(10000くらいでも収束する)
- train_crop_size：データセットの解像度(257,257 or 513,513が妥当)
- tf_initial_checkpoint：学習済み結合荷重(ダウンロードした3つから適宜選定)
- train_batch_size：ミニバッチサイズ(Colab環境だと2で限界…？)

In [ ]:
!python train.py \
  --logtostderr \
  --training_number_of_steps=$NUM_ITERATIONS \
  --train_split="train" \
  --model_variant="xception_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --train_crop_size="257,257" \
  --dataset="pascal_voc_seg" \
  --tf_initial_checkpoint=$INIT_FOLDER/xception_65_coco_pretrained/x65-b2u1s2p-d48-2-3x256-sc-cr300k_init.ckpt \
  --train_logdir=$TRAIN_LOGDIR \
  --fine_tune_batch_norm=false \
  --dataset_dir=$PASCAL_DATASET \
  --train_batch_size=2 \
  --initialize_last_layer=false \
  --last_layers_contain_logits_only=true \
  --base_learning_rate=0.001 \
  --learning_rate_decay_factor=0.3 \
  --learning_rate_decay_step=2000

# 結果

学習結果に対しmIoUを算出します．

In [ ]:
%time !python eval.py \
 --logtostderr \
 --eval_split="val" \
 --model_variant="xception_65" \
 --atrous_rates=6 \
 --atrous_rates=12 \
 --atrous_rates=18 \
 --output_stride=16 \
 --decoder_output_stride=4 \
 --eval_crop_size="257,257" \
 --checkpoint_dir=$TRAIN_LOGDIR \
 --eval_logdir=$EVAL_LOGDIR \
 --dataset_dir=$PASCAL_DATASET \
 --max_number_of_evaluations=1

学習結果に対し検証データの出力結果を可視化します．

In [ ]:
%time !python vis.py \
 --logtostderr \
 --vis_split="val" \
 --model_variant="xception_65" \
 --atrous_rates=6 \
 --atrous_rates=12 \
 --atrous_rates=18 \
 --output_stride=16 \
 --decoder_output_stride=4 \
 --vis_crop_size="257,257" \
 --checkpoint_dir=$TRAIN_LOGDIR \
 --vis_logdir=$VIS_LOGDIR \
 --dataset_dir=$PASCAL_DATASET \
 --max_number_of_iterations=1

# モデルの保存

学習モデルを保存します．

In [ ]:
CKPT_PATH = f"{TRAIN_LOGDIR}/model.ckpt-{str(NUM_ITERATIONS)}"
EXPORT_PATH = f"{EXPORT_DIR}/checkpoint_{str(NUM_ITERATIONS)}.pb"

%time !python export_model.py \
  --logtostderr \
  --checkpoint_path=$CKPT_PATH \
  --export_path=$EXPORT_PATH \
  --model_variant="xceptions_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --num_classes=21 \
  --crop_size=257 \
  --crop_size=257 \
  --inference_scales=1.0